In [13]:
import pandas as pd
import os
import numpy as np

In [6]:
def load_data(file_path):
    if not os.path.exists(file_path):
        print(f"Помилка: Файл {file_path} не знайдено!")
        return None

    ext = os.path.splitext(file_path)[-1].lower()

    try:
        if ext == '.xlsx':
            df = pd.read_excel(file_path)
        else:
            df = pd.read_csv(file_path)

        print(f"Успішно завантажено! Рядків: {df.shape[0]}, Колонок: {df.shape[1]}")
        return df
    except Exception as e:
        print(f"Не вдалося прочитати файл: {e}")
        return None


In [7]:
df = load_data("nuclear_safety_q4_2025.xlsx")
df.head()

Успішно завантажено! Рядків: 128, Колонок: 16


,year,quarter,station,irg,irg_index,iodine_ radionuclides,iodine_ radionuclides_index,stable_radionuclides,stable_ radionuclides_index,cs_137_emission,co_60_ emission,cs_137_dump,co_60_dump,volume,index_radioactive_releas,index_dump
0,2018,1,ЗАЕС,89.0,0.13,260.0,"<0,01",650.0,0.03,1980.0,1020.0,4330.0,3670.0,833000.0,0.149,0.33
1,2018,1,РАЕС,105.0,0.16,147.0,"<0,01",269.0,0.07,587.0,165.0,4800.0,620.0,2220000.0,0.780,0.096
2,2018,1,ЮУАЕС,45.0,0.10,76.0,"<0,01",116.0,0.02,136.0,373.0,390.0,370.0,14600.0,0.136,0.284
3,2018,1,ХАЕС,31.0,0.07,26.8,"<0,01",37.5,"<0,01",29.4,13.8,380.0,NaN,22070.0,0.110,0.03
4,2018,2,ЗАЕС,84.0,0.12,262.0,"<0,01",640.0,0.03,453.0,1003.0,4627.0,3432.0,812667.0,0.115,0.91


In [8]:
df.columns = [col.strip() for col in df.columns]

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128 entries, 0 to 127
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   year                         128 non-null    int64  
 1   quarter                      128 non-null    int64  
 2   station                      128 non-null    object 
 3   irg                          115 non-null    float64
 4   irg_index                    115 non-null    float64
 5   iodine_ radionuclides        115 non-null    float64
 6   iodine_ radionuclides_index  115 non-null    object 
 7   stable_radionuclides         115 non-null    float64
 8   stable_ radionuclides_index  115 non-null    object 
 9   cs_137_emission              115 non-null    float64
 10  co_60_ emission              115 non-null    float64
 11  cs_137_dump                  115 non-null    float64
 12  co_60_dump                   98 non-null     float64
 13  volume              

In [10]:
df['station'] = df['station'].replace('ЮУАЕС', 'ПАЕС')

In [11]:
def fix_numbers(val):
    if pd.isna(val): return np.nan
    s = str(val).replace(',', '.').strip()
    if '<' in s:
        return float(s.replace('<', '')) - 0.001
    try:
        return float(s)
    except:
        return np.nan

In [14]:
numeric_cols = [c for c in df.columns if c not in ['year', 'quarter', 'station']]
for col in numeric_cols:
    df[col] = df[col].apply(fix_numbers)

In [15]:
print(df.isna().sum())

year                            0
quarter                         0
station                         0
irg                            13
irg_index                      13
iodine_ radionuclides          13
iodine_ radionuclides_index    13
stable_radionuclides           13
stable_ radionuclides_index    13
cs_137_emission                13
co_60_ emission                13
cs_137_dump                    13
co_60_dump                     30
volume                         13
index_radioactive_releas       13
index_dump                     13
dtype: int64


In [16]:
print(f"Список станцій: {df['station'].unique()}")

Список станцій: ['ЗАЕС' 'РАЕС' 'ПАЕС' 'ХАЕС']


In [17]:
df

,year,quarter,station,irg,irg_index,iodine_ radionuclides,iodine_ radionuclides_index,stable_radionuclides,stable_ radionuclides_index,cs_137_emission,co_60_ emission,cs_137_dump,co_60_dump,volume,index_radioactive_releas,index_dump
0,2018,1,ЗАЕС,89.0,0.13,260.0,0.009,650.0,0.030,1980.0,1020.0,4330.0,3670.0,833000.0,0.149,0.330
1,2018,1,РАЕС,105.0,0.16,147.0,0.009,269.0,0.070,587.0,165.0,4800.0,620.0,2220000.0,0.780,0.096
2,2018,1,ПАЕС,45.0,0.10,76.0,0.009,116.0,0.020,136.0,373.0,390.0,370.0,14600.0,0.136,0.284
3,2018,1,ХАЕС,31.0,0.07,26.8,0.009,37.5,0.009,29.4,13.8,380.0,NaN,22070.0,0.110,0.030
4,2018,2,ЗАЕС,84.0,0.12,262.0,0.009,640.0,0.030,453.0,1003.0,4627.0,3432.0,812667.0,0.115,0.910
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,2025,3,ХАЕС,30.0,0.07,36.0,0.009,37.0,0.009,31.0,24.0,176.0,11.0,20008.0,0.840,0.098
124,2025,4,ЗАЕС,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125,2025,4,РАЕС,131.0,0.22,17.0,0.009,256.0,0.060,225.0,77.0,3743.0,855.0,2903600.0,0.380,0.430
126,2025,4,ПАЕС,40.0,0.09,67.0,0.009,100.0,0.010,23.0,77.0,159.0,154.0,6200.0,4.193,2.868


In [18]:
df = df.drop(columns=['co_60_dump'])

In [22]:
print(df.isna().sum())

year                           0
quarter                        0
station                        0
irg                            0
irg_index                      0
iodine_ radionuclides          0
iodine_ radionuclides_index    0
stable_radionuclides           0
stable_ radionuclides_index    0
cs_137_emission                0
co_60_ emission                0
cs_137_dump                    0
volume                         0
index_radioactive_releas       0
index_dump                     0
dtype: int64


In [21]:
df = df.dropna()

In [23]:
df

,year,quarter,station,irg,irg_index,iodine_ radionuclides,iodine_ radionuclides_index,stable_radionuclides,stable_ radionuclides_index,cs_137_emission,co_60_ emission,cs_137_dump,volume,index_radioactive_releas,index_dump
0,2018,1,ЗАЕС,89.0,0.13,260.0,0.009,650.0,0.030,1980.0,1020.0,4330.0,833000.0,0.149,0.330
1,2018,1,РАЕС,105.0,0.16,147.0,0.009,269.0,0.070,587.0,165.0,4800.0,2220000.0,0.780,0.096
2,2018,1,ПАЕС,45.0,0.10,76.0,0.009,116.0,0.020,136.0,373.0,390.0,14600.0,0.136,0.284
3,2018,1,ХАЕС,31.0,0.07,26.8,0.009,37.5,0.009,29.4,13.8,380.0,22070.0,0.110,0.030
4,2018,2,ЗАЕС,84.0,0.12,262.0,0.009,640.0,0.030,453.0,1003.0,4627.0,812667.0,0.115,0.910
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,2025,3,ПАЕС,25.0,0.06,73.0,0.009,112.0,0.010,41.0,148.0,878.0,34500.0,4.267,2.787
123,2025,3,ХАЕС,30.0,0.07,36.0,0.009,37.0,0.009,31.0,24.0,176.0,20008.0,0.840,0.098
125,2025,4,РАЕС,131.0,0.22,17.0,0.009,256.0,0.060,225.0,77.0,3743.0,2903600.0,0.380,0.430
126,2025,4,ПАЕС,40.0,0.09,67.0,0.009,100.0,0.010,23.0,77.0,159.0,6200.0,4.193,2.868
